In [1]:
import torch
from torch.optim import Adam
from models import DeepVGAE
from torch_geometric.utils import negative_sampling
from torch_geometric.datasets.planetoid import Planetoid
import torch_geometric.transforms as T
from torch_geometric.utils import train_test_split_edges
import os
import numpy as np

device_string = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_string)

seed = 42
dataset_name = "cora"
gnn = 'PEG'

torch.manual_seed(seed)
np.random.seed(seed)

os.makedirs("datasets", exist_ok=True)
dataset = Planetoid("datasets", dataset_name, transform=T.NormalizeFeatures())
data = dataset[0]#.to(device)
all_edge_index = data.edge_index
data = train_test_split_edges(data, 0.05, 0.1)

data = data.to(device)

/home/erik/Downloads/lepra/.env/lib/python3.10/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


In [2]:
if gnn == 'VGAE':
    enc_in_channels = data.x.shape[1]
    enc_hidden_channels = 32
    enc_out_channels = 16

    lr = 0.01
    epochs = 400
    model = DeepVGAE(enc_in_channels, enc_hidden_channels, enc_out_channels).to(device)
    optimizer = Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        loss = model.loss(data.x, data.train_pos_edge_index, all_edge_index)
        loss.backward()
        optimizer.step()
        if (epoch+1) % 10 == 0:
            model.eval()
            roc_auc, ap = model.single_test(data.x,
                                            data.train_pos_edge_index,
                                            data.test_pos_edge_index,
                                            data.test_neg_edge_index)
            print("Epoch {} - Loss: {} ROC_AUC: {} Precision: {}".format(epoch+1, loss.cpu().item(), roc_auc, ap))

elif gnn == 'PEG':
    #Build train matrix for PE preparation
    from torch_geometric.utils import to_networkx

    import copy
    train_graph = copy.deepcopy(data)
    train_graph.edge_index = data.train_pos_edge_index
    G = to_networkx(train_graph)
    del train_graph

    from peg_utils import *

    model_emb = DeepWalk(G,walk_length=80, num_walks=10,workers=1)#init model
    model_emb.train(embed_size = 10)# train model
    emb = model_emb.get_embeddings()# get embedding vectors
    embeddings = []
    for i in range(len(emb)):
        embeddings.append(emb[i])
    embeddings = np.array(embeddings)

    x = data.x
    pos = torch.tensor(embeddings)
    pos = pos.type(torch.FloatTensor)
    x = x.type(torch.FloatTensor)

    for run in range(1):
        total_class = 16

        model = Net(in_feats_dim = data.num_features, pos_dim = 128, hidden_dim = total_class, node_dim=data.x.shape[1])
        data = data.to(device) 
        model = model.to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr= 0.01)
        print(f'total parameters: {sum(p.numel() for p in model.parameters())}')

        best_val_perf = test_perf = 0
        for epoch in range(1, 401):
            train_loss = train_peg(model, optimizer, pos, data, x, device)
            val_perf, tmp_test_perf = test_peg(model, pos, data, x, device)
            if val_perf > best_val_perf:
                best_val_perf = val_perf
                test_perf = tmp_test_perf
            log = 'Epoch: {:03d}, Loss: {:.4f}, Val: {:.4f}, Test: {:.4f}'
            if epoch % 10 == 0:
                print(log.format(epoch, train_loss, best_val_perf, test_perf))
else:
    print('ERROR: GNN don´t exist')

Learning embedding vectors...
Learning embedding vectors done!
total parameters: 23413
Epoch: 010, Loss: 9.1198, Val: 0.1775, Test: 0.1115
Epoch: 020, Loss: 7.0931, Val: 0.1775, Test: 0.1115
Epoch: 030, Loss: 5.1068, Val: 0.1938, Test: 0.1241
Epoch: 040, Loss: 3.1091, Val: 0.2622, Test: 0.2006
Epoch: 050, Loss: 1.2857, Val: 0.6040, Test: 0.6238
Epoch: 060, Loss: 0.4644, Val: 0.7972, Test: 0.8486
Epoch: 070, Loss: 0.3294, Val: 0.8216, Test: 0.8687
Epoch: 080, Loss: 0.2533, Val: 0.8378, Test: 0.8821
Epoch: 090, Loss: 0.2160, Val: 0.8424, Test: 0.8822
Epoch: 100, Loss: 0.2104, Val: 0.8456, Test: 0.8851
Epoch: 110, Loss: 0.1948, Val: 0.8508, Test: 0.8926
Epoch: 120, Loss: 0.1872, Val: 0.8542, Test: 0.8972
Epoch: 130, Loss: 0.1760, Val: 0.8556, Test: 0.8990
Epoch: 140, Loss: 0.1693, Val: 0.8563, Test: 0.8995
Epoch: 150, Loss: 0.1632, Val: 0.8570, Test: 0.9001
Epoch: 160, Loss: 0.1582, Val: 0.8578, Test: 0.9010
Epoch: 170, Loss: 0.1520, Val: 0.8584, Test: 0.9016
Epoch: 180, Loss: 0.1437, Val

In [3]:
path="models/"
torch.save(
    {
            "model_state": model.state_dict(),
            "optimizer_state": optimizer.state_dict(),
        },
        path + gnn + '_'+dataset_name+ '_seed_' + str(seed) + '.pth.tar',)